In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Requirements


In [4]:
!unzip /content/drive/MyDrive/HIISS/Causal-Unsupervised-Segmentation.zip &> /dev/null
!cp -r /content/content/Causal-Unsupervised-Segmentation /content
!rm -r /content/content

In [7]:
!pip install -r /content/Causal-Unsupervised-Segmentation/requirements.txt &> /dev/null
!pip install timm==0.9.5 &> /dev/null
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git &> /dev/null

## 1.1 Get DINO checkpoint

In [10]:
!mkdir checkpoint
!wget https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth &> /dev/null
!mv /content/dino_deitsmall8_pretrain.pth /content/checkpoint/dino_vit_small_8.pth

# 2 Train with four classes

## 2.1 Load pirelli dataset

Choose between strided and not strided

In [5]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_4cat_strided.zip -d /content &> /dev/null

## 2.2 CAUSE Super Crop


In [8]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_4cat" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 258/258 [00:53<00:00,  4.87it/s]


## 2.3 Train the mediator
Namely get the clusterbook

In [11]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
                          --data_dir "Pirelli_4cat" --dataset "pirelli" \
                          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
                          --gpu 0 --epoch 5

------------------Configurations------------------
epoch: 5
distributed: True
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
No File Exists!!
-------------TRAIN EPOCH: 1-------------
[Train]: 100% 807/807 [03:51<00:00,  3.49it/s]
-------------TRAIN EPOCH: 2-------------
[Train]: 100% 807/807 [03:55<00:00,  3.43it/s]
-------------TRAIN EPOCH: 3-------------
[Train]: 100% 807/807 [03:54<00:00,  3.44it/s]
-------------TRAIN EPOCH: 4-------------
[Train]: 100% 807/807 [03:55<00:00,  3.43it/s]
-------------TRAIN EPOCH: 5-------------
[Train]: 100% 807/807 [03:54<00:00,  3.44it/s]


## 2.4 Train the transformer segmentator.

The clusterbook is fixed. Now, we train the transformer segmentator

In [12]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
          --gpu 0 --epoch 5 --batch_size 16

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 5
distributed: False
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 7.19=6.88+0.31 ACC: 89.4%: 100% 807/807 [08:04<00:00,  1.67it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 86.8%: 100% 6/6 [00:02<00:00,  2.44it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT IN CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth-----------------
-------------TRAIN EPOCH: 2----------

## 2.5 Fine tuning

Now, we fine tune the transformer and the clustebook. Twice following empirical results.

In [13]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 2 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 3
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.07=0.06-0.13 ACC: 97.9%: 100% 807/807 [05:41<00:00,  2.37it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 89.7% | [mIoU]: 24.6, [mAP]: 35.7, [Acc]: 53.2, : 100% 6/6 [00:02<00:00,  2.22it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

### Phase 2 retrain using the old cluster book.

In [14]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.12=0.03-0.16 ACC: 98.6%: 100% 807/807 [05:42<00:00,  2.36it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 91.5% | [mIoU]: 47.8, [mAP]: 64.1, [Acc]: 88.7, : 100% 6/6 [00:02<00:00,  2.31it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

## 2.6 Test the model

In [15]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_4cat" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test without CRF
[TEST] Acc (Linear): 91.5% | [mIoU]: 47.8, [mAP]: 64.1, [Acc]: 88.7, : 100% 6/6 [00:03<00:00,  1.80it/s]
OrderedDict([('mIoU', tensor([91.8793, 50.7347, 12.5117, 36.1078], device='cuda:0')), ('mAP', tensor(

## 2.7 Save Results and checkpoints

In [16]:
!zip -r results_only_sup_4classes_strided.zip ./results &> /dev/null

In [17]:
!zip -r /content/CAUSE_only_sup_4classes_strided.zip ./CAUSE &> /dev/null

In [19]:
!mv /content/results_only_sup_4classes_strided.zip /content/drive/MyDrive/HIISS/results
!mv /content/CAUSE_only_sup_4classes_strided.zip /content/drive/MyDrive/HIISS/checkpoints

# 3 Train with three classes




In [27]:
!rm -r /content/CAUSE # If retrain all

## 3.1 Load Pirelli 3 classes dataset

In [20]:
!unzip /content/drive/MyDrive/PROGETTO-PIRELLI/datasets/Pirelli_3cat_no_stride.zip -d /content &> /dev/null

## 3.2 Crop dataset

In [21]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py --data_dir "Pirelli_3cat" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 87/87 [00:17<00:00,  5.03it/s]


## 3.3 Train Mediator

In [29]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
                          --data_dir "Pirelli_3cat" --dataset "pirelli" \
                          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
                          --gpu 0 --epoch 3

------------------Configurations------------------
epoch: 3
distributed: True
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
No File Exists!!
-------------TRAIN EPOCH: 1-------------
[Train]: 100% 272/272 [01:20<00:00,  3.39it/s]
-------------TRAIN EPOCH: 2-------------
[Train]: 100% 272/272 [01:18<00:00,  3.45it/s]
-------------TRAIN EPOCH: 3-------------
[Train]: 100% 272/272 [01:18<00:00,  3.44it/s]


## 3.4 Train Transformer

In [30]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
          --gpu 0 --epoch 4 --batch_size 16

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 4
distributed: False
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 6.78=6.53+0.25 ACC: 92.2%: 100% 272/272 [02:40<00:00,  1.69it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 91.8%: 100% 3/3 [00:01<00:00,  2.34it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT IN CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth-----------------
-------------TRAIN EPOCH: 2----------

## 3.5 Fine-tuning

In [31]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 2 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 2
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 0.00=0.07-0.07 ACC: 97.3%: 100% 272/272 [01:57<00:00,  2.32it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 95.0% | [mIoU]: 80.8, [mAP]: 85.9, [Acc]: 95.2, : 100% 3/3 [00:01<00:00,  1.87it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT

## 3.6 Retrain with the old clusterbook

In [32]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.02=0.04-0.06 ACC: 98.2%: 100% 272/272 [01:56<00:00,  2.34it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 96.5% | [mIoU]: 83.0, [mAP]: 88.7, [Acc]: 95.8, : 100% 3/3 [00:01<00:00,  1.92it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

# Test Model

In [33]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_3cat" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test without CRF
[TEST] Acc (Linear): 96.5% | [mIoU]: 83.0, [mAP]: 88.7, [Acc]: 95.8, : 100% 3/3 [00:01<00:00,  1.54it/s]
OrderedDict([('mIoU', tensor([94.8309, 92.6635, 61.5969], device='cuda:0')), ('mAP', tensor([97.3551,

# Save Results and checkpoints

In [34]:
!zip -r results_only_sup_3classes_no_stride.zip ./results &> /dev/null

In [35]:
!zip -r /content/CAUSE_only_sup_3classes_no_stride.zip ./CAUSE &> /dev/null

In [36]:
!mv /content/CAUSE_only_sup_3classes_no_stride.zip /content/drive/MyDrive/HIISS/checkpoints
!mv results_only_sup_3classes_no_stride.zip /content/drive/MyDrive/HIISS/results

# Predict

In [37]:
!unzip /content/drive/MyDrive/PROGETTO-PIRELLI/datasets/Pirelli_unsupervised.zip -d /content &> /dev/null

In [39]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_unsupervised" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 519/519 [04:03<00:00,  2.13it/s]


In [40]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_unsupervised" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True --just_imgs True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_unsupervised
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: True
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test
100% 19/19 [04:48<00:00, 15.19s/it]


In [41]:
!zip -r results_only_sup_all_imgs.zip ./results &> /dev/null

In [42]:
!mv results_only_sup_all_imgs.zip /content/drive/MyDrive/HIISS/results